In [8]:
import os
from PIL import Image
import random
import cv2
import numpy as np
from tqdm import tqdm

In [9]:
processed_dir = r'E:\Projects\Wound-Classification-DL\data\processed'

if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
    print(f'Directory created: {processed_dir}')
else:
    print(f'Directory already exists: {processed_dir}')

Directory already exists: E:\Projects\Wound-Classification-DL\data\processed


In [10]:
target_size = (128, 128)

dataset_path = r'E:\Projects\Wound-Classification-DL\data\Categorized Wound Images Dataset'
folders = [folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))]

for folder in folders:
    folder_path = os.path.join(dataset_path, folder)
    output_folder = os.path.join(processed_dir, folder)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in os.listdir(folder_path):
        input_path = os.path.join(folder_path, file)
        output_path = os.path.join(output_folder, file)
        
        try:
            with Image.open(input_path) as img:
                resized_img = img.resize(target_size)
                resized_img.save(output_path)
        except Exception as e:
            print(f'Error processing {input_path}: {e}')

In [11]:
def augment_image(img):
    transformations = [
        lambda x: cv2.rotate(x, cv2.ROTATE_90_CLOCKWISE),
        lambda x: cv2.flip(x, 1),
        lambda x: cv2.GaussianBlur(x, (5, 5), 0),
        lambda x: cv2.cvtColor(x, cv2.COLOR_BGR2HSV),
        lambda x: shift_image(x, shift_x=10, shift_y=10),
    ]
    transform = random.choice(transformations)
    augmented = transform(img)
    if len(augmented.shape) == 3 and augmented.shape[2] == 3:
        return augmented
    else:
        return img

def shift_image(img, shift_x, shift_y):
    rows, cols, _ = img.shape
    M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
    shifted = cv2.warpAffine(img, M, (cols, rows), borderMode=cv2.BORDER_REFLECT_101)
    return shifted

def balance_dataset(input_dir, output_dir, target_size):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for folder in tqdm(os.listdir(input_dir)):
        folder_path = os.path.join(input_dir, folder)
        output_folder = os.path.join(output_dir, folder)
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        images = os.listdir(folder_path)
        num_images = len(images)
        
        for img in images:
            input_path = os.path.join(folder_path, img)
            output_path = os.path.join(output_folder, img)
            image = cv2.imread(input_path)
            cv2.imwrite(output_path, image)
        
        while len(os.listdir(output_folder)) < target_size:
            img = random.choice(images)
            input_path = os.path.join(folder_path, img)
            image = cv2.imread(input_path)
            augmented_image = augment_image(image)
            output_path = os.path.join(output_folder, f'aug_{random.randint(10000, 99999)}.jpg')
            cv2.imwrite(output_path, augmented_image)

input_dir = r'E:\Projects\Wound-Classification-DL\data\processed'
output_dir = r'E:\Projects\Wound-Classification-DL\data\balanced'
target_size = 1000

balance_dataset(input_dir, output_dir, target_size)

100%|██████████| 10/10 [00:31<00:00,  3.19s/it]
